In [1]:
import pandas as pd
import re
from datetime import datetime

## veic_2019_02.csv_amostra

In [35]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,1,501,1,2019-02-01 00:00:07,2019-02-01,1,2,2019,00:00:07,0,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
1,2,502,2,2019-02-01 00:00:09,2019-02-01,1,2,2019,00:00:09,0,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
2,3,502,2,2019-02-01 00:00:17,2019-02-01,1,2,2019,00:00:17,0,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN
3,4,501,1,2019-02-01 00:00:20,2019-02-01,1,2,2019,00:00:20,0,...,73.0,2,3.8,bdac470d1,LMR8029,0,0,NaN,NaN,NaN


## movimento_OCR_JAN_2019.txt_amostra.txt

In [161]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

# Correção para linhas que começam com múltiplos espaços
def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)
            
# Separando as colunas manualmente
df_ocr['x'] = df_ocr.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_ocr['x'] = df_ocr.x.apply(lambda x: correctSplit(x))
df_ocr['placa'] = df_ocr.x.apply(lambda x: x.split('|')[0])
df_ocr['b'] = df_ocr.x.apply(lambda x: x.split('|')[1])
df_ocr['c'] = df_ocr.x.apply(lambda x: x.split('|')[2])
df_ocr['bairro'] = df_ocr.x.apply(lambda x: x.split('|')[3])
df_ocr['e'] = df_ocr.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_ocr.drop('x', axis = 1, inplace = True)

df_ocr.sample(3)

,placa,b,c,bairro,e
42,KXK44981AUTOMÓVEL,20190101000017901034,0101201900001702900CET8-5415 0410351111025201...,St Sconrado,C-23.01567-43.41202
50,KNH04281AUTOMÓVEL,20190101000020431014,0101201900002003900CET8-5431 0410441111025201...,St Est Campinho,A-22.88462-43.57968
17,KRU31651AUTOMÓVEL,20190101000006581014,0101201900000603300CET8-5455 0410191111025201...,St R Catete,A-22.93380-43.18294


### Tratando

In [162]:
def get_datahora(string):
    date = string[:14]
    return datetime.strptime(date, "%Y%m%d%H%M%S")

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_latitude(string):
    lat = string[1:10]
    return lat

def get_longitude(string):
    lon = string[10:19]
    return lon

df_ocr['DATAHORA'] = df_ocr['b'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr['placa'].apply(lambda x: get_placa(x))
df_ocr['LATITUDE'] = df_ocr['e'].apply(lambda x: get_latitude(x))
df_ocr['LONGITUDE'] = df_ocr['e'].apply(lambda x: get_longitude(x))
df_ocr['b'] = df_ocr['b'].apply(lambda x: x[14:])
df_ocr['c'] = df_ocr['c'].apply(lambda x: x[14:])
df_ocr.drop(['placa', 'bairro', 'e'], axis =1, inplace = True )
df_ocr['CGCT'] = df_ocr['c'].apply(lambda x: x.split()[1][:3])
df_ocr['CGPT'] = df_ocr['c'].apply(lambda x: x.split()[1][3:6])
df_ocr['CGLG'] = df_ocr['c'].apply(lambda x: x.split()[1][6:7])
df_ocr['CGPI'] = df_ocr['c'].apply(lambda x: x.split()[1][7:8])
df_ocr['CGST'] = df_ocr['c'].apply(lambda x: x.split()[1][8:9])
df_ocr['CGFX'] = df_ocr['c'].apply(lambda x: x.split()[1][9:10])
df_ocr['CODCET'] = df_ocr['c'].apply(lambda x: x.split()[1][:10])
df_ocr['c'] = df_ocr['c'].apply(lambda x: x.split()[0]+' '+x.split()[1][10:])
df_ocr['VELOCIDADE'] = df_ocr['c'].apply(lambda x: x[:3]+'.'+x[3:5])
df_ocr.drop(['b', 'c', 'CGPT', 'CGPI', 'CGST', 'CGFX', 'CGLG'], axis=1, inplace=True)
df_ocr.head(10)

,DATAHORA,PLACA,LATITUDE,LONGITUDE,CGCT,CODCET,VELOCIDADE
0,2019-01-01 00:00:00,KPV6877,-22.93380,-43.18294,041,0410191112,039.00
1,2019-01-01 00:00:01,LQR8247,-22.97019,-43.41377,041,0410121122,034.00
2,2019-01-01 00:00:01,ETP1890,-22.93149,-43.20888,041,0410241111,040.00
3,2019-01-01 00:00:01,KXL9916,-22.92479,-43.38907,041,0410161111,025.00
4,2019-01-01 00:00:01,LPU2287,-23.01267,-43.38900,041,0410381122,044.00
5,2019-01-01 00:00:02,LKU5609,-23.00850,-43.43940,041,0410551112,039.00
6,2019-01-01 00:00:02,KXL9916,-22.92479,-43.38907,041,0410161121,028.00
7,2019-01-01 00:00:02,,-22.79752,-43.19290,041,0410021111,023.00
8,2019-01-01 00:00:02,,-22.98010,-43.23462,041,0410451121,035.00
9,2019-01-01 00:00:03,KPH4077,-23.01267,-43.38900,041,0410381111,038.00


## SCP_INTVIA_0182018_201901_amostra.txt

In [6]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia = pd.read_csv('SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# Separando as colunas manualmente
df_intvia['x'] = df_intvia.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_intvia['x'] = df_intvia.x.apply(lambda x: correctSplit(x))
df_intvia['veiculo'] = df_intvia.x.apply(lambda x: x.split('|')[0])
df_intvia['a'] = df_intvia.x.apply(lambda x: x.split('|')[1])
df_intvia['b'] = df_intvia.x.apply(lambda x: x.split('|')[2])
df_intvia['c'] = df_intvia.x.apply(lambda x: x.split('|')[3])
df_intvia['d'] = df_intvia.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_intvia.drop('x', axis = 1, inplace = True)

df_intvia.head()

,veiculo,a,b,c,d
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA


## SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [7]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_sitran['x'] = df_sitran.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_sitran['x'] = df_sitran.x.apply(lambda x: correctSplit(x))
df_sitran['veiculo'] = df_sitran.x.apply(lambda x: x.split('|')[0])
df_sitran['a'] = df_sitran.x.apply(lambda x: x.split('|')[1])
df_sitran['b'] = df_sitran.x.apply(lambda x: x.split('|')[2])
df_sitran['c'] = df_sitran.x.apply(lambda x: x.split('|')[3])
df_sitran['d'] = df_sitran.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_sitran.drop('x', axis = 1, inplace = True)

df_sitran.head()

,veiculo,a,b,c,d
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO


## SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [8]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_transcarioca['x'] = df_transcarioca.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_transcarioca['x'] = df_transcarioca.x.apply(lambda x: correctSplit(x))
df_transcarioca['veiculo'] = df_transcarioca.x.apply(lambda x: x.split('|')[0])
df_transcarioca['a'] = df_transcarioca.x.apply(lambda x: x.split('|')[1])
df_transcarioca['b'] = df_transcarioca.x.apply(lambda x: x.split('|')[2])
df_transcarioca['c'] = df_transcarioca.x.apply(lambda x: x.split('|')[3])
df_transcarioca['d'] = df_transcarioca.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_transcarioca.drop('x', axis = 1, inplace = True)

df_transcarioca.head()

,veiculo,a,b,c,d
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA


## SPLICE_0072018_20190205_.txt_amostra.txt

In [43]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_splice = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_splice.drop('x', axis = 1, inplace = True)

df_splice.sample(5)

,veiculo,a,b,c
65,1MOTO,2019010100045404500RD3363,04502311110072018R VOLUNTARIOS DA PATRIA PROX 143,Praia de Botafogo
5,1MOTO,2019010100002304300RD3361,04502511120072018R VOLUNTARIOS DA PATRIA PROX 301,Praia de Botafogo
12,1MOTO,2019010100004003500RD3363,04502311120072018R VOLUNTARIOS DA PATRIA PROX 143,Praia de Botafogo
27,1MOTO,2019010100015904300RD3408,04506311120072018AV ATAULFO PAIVA PX153,Copacabana
68,1MOTO,2019010100045603600RD3411,04506711120072018R VISCONDE DE PIRAJA PX39,Copacabana


## FISCALTECH_0042016_06_2019.txt_amostra.txt

In [44]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_fiscaltech['x'] = df_fiscaltech.x.replace({'( ){3,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_fiscaltech['x'] = df_fiscaltech.x.apply(lambda x: correctSplit(x))
df_fiscaltech['a'] = df_fiscaltech.x.apply(lambda x: x.split('|')[0])
df_fiscaltech['b0'] = df_fiscaltech.x.apply(lambda x: x.split('|')[1])
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: ' '.join(list(x.split(' ')[1:])))
df_fiscaltech['d'] = df_fiscaltech.x.apply(lambda x: x.split('|')[2])
# df_fiscaltech['c'] = df_fiscaltech.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_fiscaltech.drop(['x','b0'], axis = 1, inplace = True)

df_fiscaltech.sample(5)

,a,b,c,d
56,1,0106201903164307300FSCII2188,04201111120042016AvSalvAllende Px4700-PCent-S...,2-22963859-43394382
18,1,0106201906192402300FSCII2181,04200811120042016Av Americas Px900 PCent-St S...,2 -2300303 -4332414
84,1,0106201906201305700FSCII2196,04201811130042016AvAbelardoBueno Px3500 PCent...,3-22973304 -4339867
28,BXC43781,0106201903345207100FSCII2172,04200711130042016Av Americas Px2151 PCent-St ...,3 -2300141-43328091
94,LOJ21813,0106201906194601900FSCII2179,04200812130042016Av Americas Px900 PLat-St St...,3-23002958-43324151
